In [6]:
import boto3
from strands.models import BedrockModel
from botocore.config import Config as BotocoreConfig
from strands import Agent
from pydantic import BaseModel
import dotenv
import os
from strands import Agent, tool

dotenv.load_dotenv(".env", override=True)
# Create a custom boto3 session
session = boto3.Session(
    aws_access_key_id= os.environ.get('AWS_ACCESS_KEY'),
    aws_secret_access_key= os.environ.get('AWS_SECRET_KEY'),
    region_name= 'ap-southeast-1'
    
)
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)
# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="arn:aws:bedrock:ap-southeast-1:389903776084:inference-profile/apac.anthropic.claude-sonnet-4-20250514-v1:0",
    boto_session=session,
    temperature=0.3,
    top_p=0.8,
    stop_sequences=["###", "END"],
    # boto_client_config=boto_config,
)

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


In [4]:
from doc_extr import doc_extr_s3
import json 
data = doc_extr_s3(bucket="testworkflow123", prefix="pdf/vlg/")
print(json.dumps(data, indent=2, ensure_ascii=False))

[
  {
    "company": {
      "TenCongTy": "CÔNG TY CỔ PHẦN VIMC LOGISTICS",
      "MaChungKhoan": "VLG",
      "DiaChi": "Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam",
      "SoDienThoai": "04-35772047/48",
      "Email": "info@vimclogistics.vn",
      "LinhVucKinhDoanh": "Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển"
    },
    "bctc": [
      {
        "quy": "Q1/2025",
        "tong_tai_san_cuoi_quy": 236156479630,
        "loi_nhuan_sau_thue": 1116027647,
        "loi_nhuan_gop": 2304572679,
        "von_chu_so_huu": 178445053214,
        "tong_doanh_thu": 60107916752,
        "tong_tai_san": 236156479630,
        "tong_no": 57711426416,
        "gia_von_hang_ban": 57803344073,
        "loi_nhuan_gop_ve_BH_va_CCDV": 2304572679,
        "loi_nhuan_tai_chinh": 6226

In [5]:
data

({'company': {'TenCongTy': 'CÔNG TY CỔ PHẦN VIMC LOGISTICS',
   'MaChungKhoan': 'VLG',
   'DiaChi': 'Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam',
   'SoDienThoai': '04-35772047/48',
   'Email': 'info@vimclogistics.vn',
   'LinhVucKinhDoanh': 'Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển'},
  'bctc': [{'quy': 'Q1/2025',
    'tong_tai_san_cuoi_quy': 236156479630,
    'loi_nhuan_sau_thue': 1116027647,
    'loi_nhuan_gop': 2304572679,
    'von_chu_so_huu': 178445053214,
    'tong_doanh_thu': 60107916752,
    'tong_tai_san': 236156479630,
    'tong_no': 57711426416,
    'gia_von_hang_ban': 57803344073,
    'loi_nhuan_gop_ve_BH_va_CCDV': 2304572679,
    'loi_nhuan_tai_chinh': 622690374,
    'loi_nhuan_truoc_thue': 1402037633,
    'tong_tai_san_luu_dong_ngan_han': 17237452

In [7]:
structure_agent = Agent(model =bedrock_model)

In [12]:
from dataclasses import dataclass, asdict
from typing import List
@dataclass
class CompanyInfo:
    ten_cty: str
    ma_ck: str
    dia_chi: str
    so_dt: str
    email: str
    linhvuckinhdoanh: str

@dataclass
class BctcItem:
    quy: str
    tong_tai_san_cuoi_quy: int
    loi_nhuan_sau_thue: int
    loi_nhuan_gop: int
    tong_doanh_thu: int
    tong_tai_san: int
    tong_no: int
    gia_von_hang_ban: int
    loi_nhuan_gop_ve_BH_va_CCDV: int
    loi_nhuan_tai_chinh: int
    loi_nhuan_truoc_thue: int
    tong_tai_san_luu_dong_ngan_han: int
    no_ngan_han: int


class CongTy(BaseModel):
    status: bool
    error: str
    companyInfo: CompanyInfo
    bctc: List[BctcItem]
out = json.dumps(data, ensure_ascii=False)
cty_extract = structure_agent.structured_output(CongTy, f"""
Hãy phân tích và sắp xếp dữ liệu lại đặc biệt về dữ liệu báo cáo tài chính, sắp xếp lại theo list các quý ứng với các biến sắp xếp theo chiều tăng dần về thời gian.
Trả về status: False khi không trích xuất được đủ trường cho 4 quý, và error khi status là False sẽ chỉ ra các trường còn thiếu. 
{out}
""")

Tôi sẽ phân tích và sắp xếp dữ liệu báo cáo tài chính của công ty VIMC LOGISTICS theo thứ tự thời gian tăng dần.
Tool #3: CongTy


In [14]:
cty_extract.companyInfo

CompanyInfo(ten_cty='CÔNG TY CỔ PHẦN VIMC LOGISTICS', ma_ck='VLG', dia_chi='Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam', so_dt='04-35772047/48', email='info@vimclogistics.vn', linhvuckinhdoanh='Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển')

In [15]:
json.dumps(asdict(cty_extract.companyInfo), ensure_ascii=False)

'{"ten_cty": "CÔNG TY CỔ PHẦN VIMC LOGISTICS", "ma_ck": "VLG", "dia_chi": "Tầng 4, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam", "so_dt": "04-35772047/48", "email": "info@vimclogistics.vn", "linhvuckinhdoanh": "Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển"}'

In [4]:
output = {'company': {'TenCongTy': 'CÔNG TY CỔ PHẦN VIMC LOGISTICS',
  'MaChungKhoan': 'VLG',
  'DiaChi': 'Phòng 806, tòa nhà Ocean Park, số 1 Đào Duy Anh, phường Phương Mai, quận Đống Đa, TP.Hà Nội, Việt Nam',
  'SoDienThoai': '024-35772047/48',
  'Email': 'info@vimclogistics.vn',
  'LinhVucKinhDoanh': 'Kinh doanh xuất nhập khẩu hàng hóa, môi giới thương mại, đại lý mua bán, ký gửi hàng hóa; Vận tải đa phương thức; vận tải hàng hóa, container, hàng hóa siêu trường, siêu trọng bằng đường bộ, đường sắt, đường biển'},
 'bctc': [{'quy': 'Q1/2025',
   'tong_tai_san_cuoi_quy': 236156479630,
   'loi_nhuan_sau_thue': 1116027647,
   'loi_nhuan_gop': 2304572679,
   'von_chu_so_huu': 178445053214,
   'tong_doanh_thu': 60107916752,
   'tong_tai_san': 236156479630,
   'tong_no': 57711426416,
   'gia_von_hang_ban': 57803344073,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2304572679,
   'loi_nhuan_tai_chinh': 622690374,
   'loi_nhuan_truoc_thue': 1402037633,
   'tong_tai_san_luu_dong_ngan_han': 172374527886,
   'no_ngan_han': 57711426416},
  {'quy': 'Q2/2024',
   'tong_tai_san_cuoi_quy': 212146243168,
   'loi_nhuan_sau_thue': 10588556972,
   'loi_nhuan_gop': 2283265819,
   'von_chu_so_huu': 167262603675,
   'tong_doanh_thu': 42083671325,
   'tong_tai_san': 212146243168,
   'tong_no': 44883639493,
   'gia_von_hang_ban': 39800405506,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2283265819,
   'loi_nhuan_tai_chinh': 190238689,
   'loi_nhuan_truoc_thue': 13259726458,
   'tong_tai_san_luu_dong_ngan_han': 145623535885,
   'no_ngan_han': 44883639493},
  {'quy': 'Q3/2024',
   'tong_tai_san_cuoi_quy': 227375689868,
   'loi_nhuan_sau_thue': 13875548792,
   'loi_nhuan_gop': 1791245409,
   'von_chu_so_huu': 181138152467,
   'tong_doanh_thu': 41977528911,
   'tong_tai_san': 227375689868,
   'tong_no': 46237537401,
   'gia_von_hang_ban': 40186283502,
   'loi_nhuan_gop_ve_BH_va_CCDV': 1791245409,
   'loi_nhuan_tai_chinh': 834803115,
   'loi_nhuan_truoc_thue': 17204922944,
   'tong_tai_san_luu_dong_ngan_han': 162255806260,
   'no_ngan_han': 46237537401},
  {'quy': 'Q4/2024',
   'tong_tai_san_cuoi_quy': 229773312284,
   'loi_nhuan_sau_thue': -3809126900,
   'loi_nhuan_gop': 2114408986,
   'von_chu_so_huu': 177329025567,
   'tong_doanh_thu': 62287700880,
   'tong_tai_san': 229773312284,
   'tong_no': 52444286717,
   'gia_von_hang_ban': 60173291894,
   'loi_nhuan_gop_ve_BH_va_CCDV': 2114408986,
   'loi_nhuan_tai_chinh': 617879595,
   'loi_nhuan_truoc_thue': -513742519,
   'tong_tai_san_luu_dong_ngan_han': 164800586261,
   'no_ngan_han': 52444286717}]}

In [2]:
agent = Agent(model = bedrock_model)
agent("Hello")

Hello! How are you doing today? Is there anything I can help you with?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Hello! How are you doing today? Is there anything I can help you with?'}]}, metrics=EventLoopMetrics(cycle_count=1, tool_metrics={}, cycle_durations=[1.3987114429473877], traces=[<strands.telemetry.metrics.Trace object at 0x0000021CC423DFC0>], accumulated_usage={'inputTokens': 8, 'outputTokens': 20, 'totalTokens': 28}, accumulated_metrics={'latencyMs': 1238}), state={})

In [3]:
from strands import Agent

structure_agent = Agent(model =bedrock_model)
trichxuat = structure_agent.structured_output(CongTy, f"""
Hãy phân tích và sắp xếp dữ liệu lại đặc biệt về dữ liệu báo cáo tài chính, sắp xếp lại theo list các quý ứng với các biến sắp xếp theo chiều tăng dần về thời gian.
Trả về status: False khi không trích xuất được đủ trường cho 4 quý
{json.dumps(output, ensure_ascii=False)}
                                           """)

NameError: name 'CongTy' is not defined

In [6]:
trichxuat.bctc

[BctcItem(quy='Q2/2024', tong_tai_san_cuoi_quy=212146243168, loi_nhuan_sau_thue=10588556972, loi_nhuan_gop=2283265819, tong_doanh_thu=42083671325, tong_tai_san=212146243168, tong_no=44883639493, gia_von_hang_ban=39800405506, loi_nhuan_gop_ve_BH_va_CCDV=2283265819, loi_nhuan_tai_chinh=190238689, loi_nhuan_truoc_thue=13259726458, tong_tai_san_luu_dong_ngan_han=145623535885, no_ngan_han=44883639493),
 BctcItem(quy='Q3/2024', tong_tai_san_cuoi_quy=227375689868, loi_nhuan_sau_thue=13875548792, loi_nhuan_gop=1791245409, tong_doanh_thu=41977528911, tong_tai_san=227375689868, tong_no=46237537401, gia_von_hang_ban=40186283502, loi_nhuan_gop_ve_BH_va_CCDV=1791245409, loi_nhuan_tai_chinh=834803115, loi_nhuan_truoc_thue=17204922944, tong_tai_san_luu_dong_ngan_han=162255806260, no_ngan_han=46237537401),
 BctcItem(quy='Q4/2024', tong_tai_san_cuoi_quy=229773312284, loi_nhuan_sau_thue=-3809126900, loi_nhuan_gop=2114408986, tong_doanh_thu=62287700880, tong_tai_san=229773312284, tong_no=52444286717, gia

In [7]:
from dataclasses import dataclass, asdict
json_data = [json.dumps(asdict(item), ensure_ascii=False) for item in trichxuat.bctc]

In [8]:
json_data

['{"quy": "Q2/2024", "tong_tai_san_cuoi_quy": 212146243168, "loi_nhuan_sau_thue": 10588556972, "loi_nhuan_gop": 2283265819, "tong_doanh_thu": 42083671325, "tong_tai_san": 212146243168, "tong_no": 44883639493, "gia_von_hang_ban": 39800405506, "loi_nhuan_gop_ve_BH_va_CCDV": 2283265819, "loi_nhuan_tai_chinh": 190238689, "loi_nhuan_truoc_thue": 13259726458, "tong_tai_san_luu_dong_ngan_han": 145623535885, "no_ngan_han": 44883639493}',
 '{"quy": "Q3/2024", "tong_tai_san_cuoi_quy": 227375689868, "loi_nhuan_sau_thue": 13875548792, "loi_nhuan_gop": 1791245409, "tong_doanh_thu": 41977528911, "tong_tai_san": 227375689868, "tong_no": 46237537401, "gia_von_hang_ban": 40186283502, "loi_nhuan_gop_ve_BH_va_CCDV": 1791245409, "loi_nhuan_tai_chinh": 834803115, "loi_nhuan_truoc_thue": 17204922944, "tong_tai_san_luu_dong_ngan_han": 162255806260, "no_ngan_han": 46237537401}',
 '{"quy": "Q4/2024", "tong_tai_san_cuoi_quy": 229773312284, "loi_nhuan_sau_thue": -3809126900, "loi_nhuan_gop": 2114408986, "tong_do

In [9]:
type(json_data)

list

In [10]:
from finance_agent import FinancialAgent

In [11]:
fin_agent = FinancialAgent()

defaulted to us-west-2 because no region was specified
issue=<https://github.com/strands-agents/sdk-python/issues/238> | this behavior will change in an upcoming release


In [ ]:
fin_data = fin_agent.compute(str(json_data))

In [15]:
type(fin_data)

str

In [18]:
class TaiChinh(BaseModel):
    status: bool
    thongso: str
    diemmanh: str
    diemyeu: str
    ruiro: str
    ketluan: str

fin_struct_agent = Agent(model = bedrock_model, system_prompt="You are an extracted agent that extract agent based on financial analyst. Status is true if result show that previous agent actually use tool")
fin_analyst = fin_struct_agent.structured_output(TaiChinh, fin_data)




Tool #1: TaiChinh


In [21]:
fin_analyst.diemmanh

'Khả năng thanh toán ngắn hạn xuất sắc với Current Ratio luôn trên 2.9. Cấu trúc vốn an toàn với tỷ lệ nợ thấp (dưới 25%). Doanh thu tăng trưởng mạnh 48% trong Q4/2024. Tổng tài sản tăng ổn định từ 212 tỷ lên 236 tỷ đồng. Vốn chủ sở hữu duy trì ổn định khoảng 178 tỷ đồng.'

In [22]:
fin_sum = f"""
Điểm mạnh: {fin_analyst.diemmanh}
Điểm yếu: {fin_analyst.diemyeu}
Rủi ro: {fin_analyst.ruiro}
"""